In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from model_mo2cap2 import mo2cap_ours
from ours_model import Ours_model
from tqdm import tqdm
import os

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = Ours_model(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		image = test_batch['image'].cuda()
		image_zoom = test_batch['image_zoom'].cuda()
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {
			'image':image,
			# 'image_zoom' : image_zoom,
			}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp818'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp818/0.pth',
 '/workspace/2d_to_3d/apps/exp818/1.pth',
 '/workspace/2d_to_3d/apps/exp818/2.pth',
 '/workspace/2d_to_3d/apps/exp818/3.pth',
 '/workspace/2d_to_3d/apps/exp818/4.pth',
 '/workspace/2d_to_3d/apps/exp818/5.pth',
 '/workspace/2d_to_3d/apps/exp818/6.pth',
 '/workspace/2d_to_3d/apps/exp818/7.pth',
 '/workspace/2d_to_3d/apps/exp818/8.pth',
 '/workspace/2d_to_3d/apps/exp818/9.pth',
 '/workspace/2d_to_3d/apps/exp818/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	if i<6:continue
	_path = f'/workspace/2d_to_3d/apps/eval/Ours_sil_dep_fusion_depthWASP818'
	_name = f'baseline818_{i}.json'
	if not os.path.exists(_path):
		os.makedirs(_path)

	main(model_path=p,output_path=os.path.join(_path,_name))

17:52:48 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 184.16763305664062, 'LowerStretching': 168.65956115722656, 'UpperStretching': 194.3202362060547, 'Talking': 146.71676635742188, 'Walking': 190.3146514892578, 'Gaming': 149.99346923828125, 'Gesticuling': 129.7270050048828, 'Reacting': 155.89378356933594, 'Patting': 155.2561798095703, 'Greeting': 139.6853790283203}, 'UpperBody': {'All': 120.37799072265625, 'LowerStretching': 100.22762298583984, 'UpperStretching': 128.1470184326172, 'Talking': 99.10234832763672, 'Walking': 124.26162719726562, 'Gaming': 100.4915542602539, 'Gesticuling': 94.87371826171875, 'Reacting': 117.59468841552734, 'Patting': 108.76888275146484, 'Greeting': 97.33226776123047}, 'LowerBody': {'All': 247.95726013183594, 'LowerStretching': 237.09152221679688, 'UpperStretching': 260.4934387207031, 'Talking': 194.33116149902344, 'Walking': 256.36767578125, 'Gaming': 199.49537658691406, 'Gesticuling': 164.58026123046875, 'Reacting': 194.19288635253906, 'Patting': 201.7435302734375, 'Greeting': 182.038467

  7%|▋         | 401/5352 [01:09<12:07,  6.80it/s]

{'FullBody': {'All': 184.43504333496094, 'LowerStretching': 168.68724060058594, 'UpperStretching': 195.3533935546875, 'Talking': 146.37294006347656, 'Walking': 190.2654571533203, 'Gaming': 148.88685607910156, 'Gesticuling': 134.11819458007812, 'Reacting': 153.7349090576172, 'Patting': 153.8046112060547, 'Greeting': 144.4341583251953}, 'UpperBody': {'All': 120.16796112060547, 'LowerStretching': 100.43462371826172, 'UpperStretching': 127.89774322509766, 'Talking': 98.6031265258789, 'Walking': 126.58515167236328, 'Gaming': 96.45831298828125, 'Gesticuling': 99.43941497802734, 'Reacting': 112.78589630126953, 'Patting': 107.4834976196289, 'Greeting': 98.79149627685547}, 'LowerBody': {'All': 248.7021026611328, 'LowerStretching': 236.93983459472656, 'UpperStretching': 262.80902099609375, 'Talking': 194.1427459716797, 'Walking': 253.94573974609375, 'Gaming': 201.3153839111328, 'Gesticuling': 168.7969207763672, 'Reacting': 194.68389892578125, 'Patting': 200.12571716308594, 'Greeting': 190.076797

 11%|█         | 601/5352 [01:38<12:35,  6.29it/s]

...

 93%|█████████▎| 5001/5352 [12:29<00:55,  6.37it/s]

{'FullBody': {'All': 81.03843688964844, 'Greeting': 47.04254150390625, 'Reacting': 64.12323760986328, 'LowerStretching': 77.17101287841797, 'Walking': 81.34384155273438, 'UpperStretching': 85.80071258544922, 'Patting': 80.79283142089844, 'Talking': 51.6414909362793, 'Gesticuling': 55.71262741088867, 'Gaming': 65.745849609375}, 'UpperBody': {'All': 59.3296012878418, 'Greeting': 36.64509582519531, 'Reacting': 56.25522994995117, 'LowerStretching': 47.92436599731445, 'Walking': 55.316932678222656, 'UpperStretching': 65.35411071777344, 'Patting': 54.649478912353516, 'Talking': 40.271568298339844, 'Gesticuling': 46.34600067138672, 'Gaming': 54.25580978393555}, 'LowerBody': {'All': 102.74727630615234, 'Greeting': 57.43998718261719, 'Reacting': 71.99124908447266, 'LowerStretching': 106.41767120361328, 'Walking': 107.37075805664062, 'UpperStretching': 106.247314453125, 'Patting': 106.93619537353516, 'Talking': 63.01141357421875, 'Gesticuling': 65.07925415039062, 'Gaming': 77.23590850830078}}


 97%|█████████▋| 5201/5352 [12:59<00:24,  6.18it/s]

{'FullBody': {'All': 81.02518463134766, 'Greeting': 47.006404876708984, 'Reacting': 64.20122528076172, 'LowerStretching': 77.08159637451172, 'Walking': 81.32328796386719, 'UpperStretching': 85.80549621582031, 'Patting': 80.75770568847656, 'Talking': 51.68637466430664, 'Gesticuling': 55.60093688964844, 'Gaming': 65.72148895263672}, 'UpperBody': {'All': 59.33051681518555, 'Greeting': 36.64870071411133, 'Reacting': 56.376399993896484, 'LowerStretching': 47.867088317871094, 'Walking': 55.27642822265625, 'UpperStretching': 65.373046875, 'Patting': 54.682708740234375, 'Talking': 40.341888427734375, 'Gesticuling': 46.24913024902344, 'Gaming': 54.12533187866211}, 'LowerBody': {'All': 102.7198715209961, 'Greeting': 57.36410903930664, 'Reacting': 72.02605438232422, 'LowerStretching': 106.29611206054688, 'Walking': 107.37015533447266, 'UpperStretching': 106.23793029785156, 'Patting': 106.83271026611328, 'Talking': 63.03086471557617, 'Gesticuling': 64.95274353027344, 'Gaming': 77.31765747070312}}


100%|██████████| 5352/5352 [13:22<00:00,  6.67it/s]
20:20:26 [INFO] Main: Saving evaluation results...
20:20:26 [INFO] Main: Done.
